## Attribute Access: \_\_getattr\_\_ and \_\_setattr\_\_

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
os.getcwd()
os.chdir('/Users/fizz/Document/Notes/Python/codes')

The **\_\_getattr\_\_** method intercepts attribute references. It's called with the attribute name as a string whenever you try to qualify an instance with an **undefined** (nonexistent) attribute name. It is **not** called if Python can find the attribute using its inheritance tree search procedure.

In [1]:
class Empty:
    def __getattr__(self, attrname):
        if attrname == 'age':
            return 40
        else:
            raise AttributeError

The **\_\_setattr\_\_** intercepts *all* attribute assignments!!! This method is a bit tricker to use, though, because assigning to any **self** attributes within **\_\_setattr\_\_** calls **\_\_setattr\_\_** again.

You can avoid loops by coding instance attribute assignments as assignments to attribute dictionary keys. That is, use **self.\_\_dict\_\_['name'] = x**, not **self.name = x**; because you're not assigning to **\_\_dict\_\_** itself, this avoid loops.

In [ ]:
class Accesscontrol:
    def __setattr__(self, attr, value):
        if attr == 'age':
            self.__dict__[attr] = value + 10    # Not self.name = val or setattr
        else:
            raise AttributeError
            
# It's also possible to avoid recursive loops in a class that uses __setattr__ by routing
# any attribute assignments to a higher superclass with a call, instead of assigning keys
# in __dict__
self.__dict__[attr] = value + 10
object.__setattr__(self, attr, value + 10)

The **\_\_getattribute\_\_** method intercepts **all** attribute fetches, not just those that are undefined, but when using it you must be more cautious than with **\_\_getattr\_\_** to avoid loops.

In [ ]:
# private0.py
class PrivateExc(Exception): pass
class Privacy:
    def __setattr__(self, attrname, value):
        if attrname in self.privates:
            raise PrivateExc(attrname, self)
        else:
            self.__dict__[attrname] = value
class Test1(Privacy):
    privates = ['age']
class Test2(Privacy):
    privates = ['name', 'pay']
    def __init__(self):
        self.__dict__['name'] = 'Tom'
if __name__ == '__main__':
    x = Test1()
    y =Test2()
    x.name = 'Bob'    # Works
    y.name = 'Sue'    # fails
    y.age = 30          # works
    x.age = 40          # fails

## String Representation: \_\_repr\_\_ and \_\_str\_\_

1. **\_\_str\_\_** is tried first for the **print** operation and the **str** built-in function (the internal equivalent of which **print** runs). It generally should return a user-friendly display.

2. **\_\_repr\_\_** is used in all other contexts: for interactive echoes, the **repr** funciton, and nested appearances, as well as by **print** and **str** if no **\_\_str\_\_** is present. It should generally return an as-code string that could be used to re-create the object, or a deatailed display for developers.

In [4]:
class adder:
    def __init__(self, value=0):
        self.data = value
    def __add__(self, other):
        self.data += other
class addboth(adder):
    def __str__(self):
        return '[Value: %s]' % self.data
    def __repr__(self):
        return 'addboth(%s)' % self.data
x = addboth(4)
x + 1
x
print(x)
str(x), repr(x)

addboth(5)

[Value: 5]


('[Value: 5]', 'addboth(5)')

First, keep in mind that **\_\_str\_\_** and **\_\_repr\_\_** must both return *strings*; other result types are not converted and raise errors, so be sure to run them through a to-string converter (e.g., **str** or %) if needed.

Second, depending on a container's string-conversion logic, the user-friendly display of **\_\_str\_\_** might only apply when objects appear at the top level of a print operation; objects *nested* in larger objects might still print with their **\_\_repr\_\_** or its result. The following illustrates both of these points:

In [4]:
class Printer:
    def __init__(self, val):
        self.val = val
    def __str__(self):
        return str(self.val)
objs = [Printer(2), Printer(3)]
for x in objs: print(x)
print(objs)
objs

2
3
[<__main__.Printer object at 0x10845a860>, <__main__.Printer object at 0x10845a4e0>]


[<__main__.Printer at 0x10845a860>, <__main__.Printer at 0x10845a4e0>]

In [5]:
# To ensure that a custom display is run in all contexts, code __repr__
class Printer:
    def __init__(self, val):
        self.val = val
    def __repr__(self):
        return str(self.val)
objs = [Printer(2), Printer(3)]
for x in objs: print(x)
print(objs)
objs

2
3
[2, 3]


[2, 3]

## Right-Side and In-Place Users: \_\_radd\_\_ and \_\_iadd\_\_

In [7]:
class Commuter1:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    def __radd__(self, other):
        print('radd', self.val, other)
        return other + self.val
x = Commuter1(88)
y = Commuter1(99)
x + 1
1 + y
x + y

add 88 1


89

radd 99 1


100

add 88 <__main__.Commuter1 object at 0x108484940>
radd 99 88


187

In [9]:
class Commuter2:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    def __radd__(self, other):
        return self.__add__(other)          # Call __add__ explictly
class Commuter3:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    def __radd__(self, other):
        return self + other                    # Swap order and re-order
class Commuter4:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    __radd__ = __add__                     # Alias: cut out the middleman

In [14]:
# Propagating class type
class Commuter5:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        if isinstance(other, Commuter5):        # Type test to avoid object nesting
            other = other.val
        return Commuter5(self.val + other)
    def __radd__(self, other):
        return Commuter5(other + self.val)
    def __str__(self):
        return '<Commuter5: %s>' % self.val
x = Commuter5(88)
y = Commuter5(99)
print(x + 10)
print(10 + y)
z = x + y
print(z)
print(z + 10)
print(z + z)
print(z + z + 1)

<Commuter5: 98>
<Commuter5: 109>
<Commuter5: 187>
<Commuter5: 197>
<Commuter5: 374>
<Commuter5: 375>


In [17]:
# With isinstance test commented-out
class Commuter5:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        return Commuter5(self.val + other)
    def __radd__(self, other):
        return Commuter5(other + self.val)
    def __str__(self):
        return '<Commuter5: %s>' % self.val
x = Commuter5(88)
y = Commuter5(99)
print(x + 10)
print(10 + y)
z = x + y
print(z)
print(z + 10)
print(z + z)
print(z + z + 1)

<Commuter5: 98>
<Commuter5: 109>
<Commuter5: <Commuter5: 187>>
<Commuter5: <Commuter5: 197>>
<Commuter5: <Commuter5: <Commuter5: <Commuter5: 374>>>>
<Commuter5: <Commuter5: <Commuter5: <Commuter5: 375>>>>


In-Place Addition

To also implement += in-place augmented addition, code either an **\_\_iadd\_\_** or an **\_\_add\_\_**. The latter is used if the former is absent.

In [7]:
class Number:
    def __init__(self, val):
        self.val = val
    def __iadd__(self, other):
        self.val += other
        return self
# For mutable objects, this method can often specialize for quicker in-place changes
y = Number([1])
y += [2]
y += [3]
y.val

[1, 2, 3]

## Call Expressions: \_\_call\_\_

In [ ]:
class Callee:
    def __call__(self, *pargs, **kargs):
        print('Called:', pargs, kargs)

## Comparisons: \_\_lt\_\_, \_\_gt\_\_, and Others

There are no implict relationships among the comparison operators. The truth of == dose not imply that != is false, for example, so both **\_\_eq\_\_** and **\_\_ne\_\_** should be defined to ensure that both operators behave correctly.

In [ ]:
class C:
    data = 'spam'
    def __gt__(self, other):
        return self.data > other
    def __lt__(self, other):
        return self.data < other

## Boolean Tests: \_\_bool\_\_ and \_\_len\_\_

In Boolean contexts, Pyton first tries **\_\_bool\_\_** to obtain a direct Boolean value; if that method is missing, Python tries **\_\_len\_\_** to infer a truth value from the object's length.

In [20]:
class Truth:
    def __bool__(self):
        return True
X = Truth()
if X: print('yes!')
    
class Truth:
    def __len__(self):
        return 0
X = Truth()
if not X: print('no!')
    
# If neither truth method is defined, the object is vacuously considered true:
class Truth:
    pass
X = Truth()
bool(X)

yes!
no!


True

## Object Destruction: **\_\_del\_\_**

In [24]:
class Life:
    def __init__(self, name='unknown'):
        print('Hello' + name)
        self.name = name
    def live(self):
        print(self.name)
    def __del__(self):
        print('Goodbye' + name)
brian = Life('Brian')
brian.live()
brian = 'loretta'

HelloBrian
Brian


Exception ignored in: <function Life.__del__ at 0x102adfc80>
Traceback (most recent call last):
  File "<ipython-input-24-21c8a8de02fa>", line 8, in __del__
NameError: name 'name' is not defined
